In [2]:
import cv2
import numpy as np
import math

In [ ]:
"""Using OpenPose, the body skeleton is extracted and by
calculating the knee joint angle, the sitting and standing
positions are detected geometrically and independently 
of the image scale."""

""""Due to the high computational cost of OpenPose, 
 frame skipping is used; the body pose detection is performed once
 every N frames and the result of the last processed frame is used for the
 intermediate frames. This provides a good balance between accuracy and speed."""

# ===================== CONFIG =====================
VIDEO_PATH = "video/Presentation1.mp4"
PROTO = "model/caffe/pose_deploy_linevec_faster_4_stages.prototxt"
WEIGHTS = "model/caffe/pose_iter_160000.caffemodel"

INPUT_W, INPUT_H = 256, 256      # smaller = faster
FRAME_SKIP = 7                # process every N frames
CONF_THRESHOLD = 0.1
KNEE_ANGLE_THRESHOLD = 150       # degrees

OUTPUT_VIDEO = "video/output_pose.mp4"
# ==================================================

# Load model
net = cv2.dnn.readNetFromCaffe(PROTO, WEIGHTS)

# Open video
cap = cv2.VideoCapture(VIDEO_PATH)
assert cap.isOpened(), "❌ Cannot open video"

fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video
out = cv2.VideoWriter(
    OUTPUT_VIDEO,
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps if fps > 0 else 10,
    (w, h)
)

# MPI model → 15 keypoints
nPoints = 15

POSE_PAIRS = [
    (1,2),(2,3),(3,4),
    (1,5),(5,6),(6,7),
    (1,8),(8,9),(9,10),
    (1,11),(11,12),(12,13)
]

# Right leg keypoints
HIP, KNEE, ANKLE = 8, 9, 10

def angle(a, b, c):
    ba = a - b
    bc = c - b
    cos_angle = np.dot(ba, bc) / (np.linalg.norm(ba)*np.linalg.norm(bc) + 1e-6)
    return np.degrees(np.arccos(np.clip(cos_angle, -1.0, 1.0)))

frame_id = 0
last_label = "Detecting..."
last_points = None

# ===================== MAIN LOOP =====================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    if frame_id % FRAME_SKIP == 0:
        blob = cv2.dnn.blobFromImage(
            frame, 1/255, (INPUT_W, INPUT_H),
            (0,0,0), swapRB=False, crop=False
        )

        net.setInput(blob)
        output = net.forward()

        points = []

        for i in range(nPoints):
            probMap = output[0, i, :, :]
            _, prob, _, point = cv2.minMaxLoc(probMap)

            x = int(w * point[0] / output.shape[3])
            y = int(h * point[1] / output.shape[2])

            if prob > CONF_THRESHOLD:
                points.append(np.array([x, y]))
            else:
                points.append(None)

        # Classification
        if points[HIP] is not None and points[KNEE] is not None and points[ANKLE] is not None:
            knee_angle = angle(points[HIP], points[KNEE], points[ANKLE])
            last_label = "Standing" if knee_angle > KNEE_ANGLE_THRESHOLD else "Sitting"
        else:
            last_label = "Unknown"

        last_points = points

    # Draw skeleton
    if last_points is not None:
        for a, b in POSE_PAIRS:
            if last_points[a] is not None and last_points[b] is not None:
                cv2.line(frame, tuple(last_points[a]), tuple(last_points[b]), (0,255,0), 2)
                cv2.circle(frame, tuple(last_points[a]), 4, (0,0,255), -1)

    # Draw label
    cv2.putText(
        frame, last_label, (30, 50),
        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255,0,0), 3
    )

    out.write(frame)
    cv2.imshow("OpenPose Sitting / Standing", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC
        break

    frame_id += 1

# ===================== CLEANUP =====================
cap.release()
out.release()
cv2.destroyAllWindows()